In [ ]:
pip install sounddevice scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyannote.audio

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install torchaudio

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyannote.audio==3.1

  Using cached pyannote.audio-3.1.0-py2.py3-none-any.whl.metadata (7.8 kB)
Using cached pyannote.audio-3.1.0-py2.py3-none-any.whl (208 kB)
  Attempting uninstall: pyannote.audio
    Found existing installation: pyannote.audio 3.3.2
    Uninstalling pyannote.audio-3.3.2:
      Successfully uninstalled pyannote.audio-3.3.2
Note: you may need to restart the kernel to use updated packages.


In [15]:
!pip install openai-whisper

In [16]:
!pip install --upgrade pyannote.audio

In [17]:
pip install --upgrade pip setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install numpy==1.23.5

Note: you may need to restart the kernel to use updated packages.Collecting numpy==1.23.5
  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'



  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\HARI\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
      main()
    File "C:\Users\HARI\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
      json_out["return_val"] = hook(**hook_input["kwargs"])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\HARI\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 137, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\HARI\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 70, in _build_backend
      obj = import_module(mod_path)
            ^^^^^^^^^^^^^^^^^^^^^^^
 

In [1]:
import sounddevice as sd
from scipy.io.wavfile import write
import torchaudio
import torch
from pyannote.audio import Pipeline
from whisper import load_model
from pyannote.core import Segment

# 1. Record audio
duration = 10  # seconds
sample_rate = 44100  # Hz
print("Recording...")
audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype='int16')
sd.wait()
print("Recording completed!")

output_file = "audio.wav"
write(output_file, sample_rate, audio)
print(f"Audio saved to {output_file}")

# 2. Load diarization pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token="replace_hf_token"
)

# 3. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)

# 4. Load recorded audio
waveform, sr = torchaudio.load(output_file)

# 5. Perform diarization
print("Running speaker diarization...")
diarization = pipeline({"waveform": waveform, "sample_rate": sr})
print("Diarization completed!")

# 6. Load Whisper model
print("Loading Whisper ASR model...")
asr_model = load_model("base")  # Options: tiny, base, small, medium, large
print("ASR model loaded!")

# 7. Extract and transcribe speaker segments
speaker_text = {}
for turn, _, speaker in diarization.itertracks(yield_label=True):
    segment = waveform[:, int(turn.start * sr): int(turn.end * sr)]
    temp_file = "temp_segment.wav"
    torchaudio.save(temp_file, segment, sr)

    result = asr_model.transcribe(temp_file)
    transcription = result["text"]

    if speaker not in speaker_text:
        speaker_text[speaker] = ""
    speaker_text[speaker] += f" {transcription}"

# 8. Print results
num_speakers = len(speaker_text)
print(f"\nNumber of speakers: {num_speakers}")
for speaker, text in speaker_text.items():
    print(f"{speaker}: {text}")


C:\Users\HARI\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Recording...
Recording completed!
Audio saved to audio.wav


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\HARI\anaconda3\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


Running speaker diarization...


C:\Users\HARI\anaconda3\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


Diarization completed!
Loading Whisper ASR model...
ASR model loaded!


C:\Users\HARI\anaconda3\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Number of speakers: 1
SPEAKER_00:   Hello and hi everyone, what's up in here? And we will do this already what goes forward.  That's why I am brutal.
